
## Imports

In [353]:
from IPython.display import display

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split #, KFold
from sklearn.metrics import roc_auc_score, root_mean_squared_error #, mutual_info_score, roc_curve, auc
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_text #, plot_tree, export_graphviz
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

from typing import Iterable, TypeVar

skPredict = TypeVar("skPredict", LogisticRegression, DecisionTreeClassifier, RandomForestClassifier)
skDecide = TypeVar("skDecide", skPredict, LinearRegression, DecisionTreeRegressor, RandomForestRegressor)
skFit = TypeVar("skFit", skPredict, skDecide)

# import matplotlib.pyplot as plt
# import seaborn as sns
# %matplotlib inline

# %pip install tqdm
from tqdm.auto import tqdm

# %pip install XGBoost
import xgboost as xgb

## Utilities

In [302]:
def numerical_features(df: pd.DataFrame):
    cols = df.columns[ (df.dtypes != 'object') ]
    return list(cols)

def categorical_features(df: pd.DataFrame):
    cols = df.columns[ (df.dtypes == 'object') ]
    return list(cols)

In [303]:
def validation_testing_training_full_split(dataframe: pd.DataFrame, seed: int = 42, validation: float = 0.2, testing: float = 0.2):
    assert 0 < validation and 0 < testing and 1 > (validation + testing)

    validation_of_full = validation / (1 - testing)
    if validation_of_full == 0:
        validation_of_full = None
        
    df_full,     df_testing    = train_test_split(dataframe, test_size=testing,            random_state=seed, shuffle=True)
    df_training, df_validation = train_test_split(df_full,   test_size=validation_of_full, random_state=seed, shuffle=True)
    
    df_validation = df_validation.reset_index(drop=True)
    df_testing = df_testing.reset_index(drop=True)
    df_training = df_training.reset_index(drop=True)
    df_full = df_full.reset_index(drop=True)
    
    return df_validation, df_testing, df_training, df_full

In [304]:
def y_split(dataframe: pd.DataFrame, yColumn: str, drop: Iterable[str] = []):
    columns = set(dataframe.columns)   
    assert columns.issuperset([yColumn]), f'{yColumn} not found in dataframe'
    assert columns.issuperset(drop), f'At least one of {drop} not found in dataframe'
    
    df = dataframe.copy()
    y = df[yColumn]
    for col in drop + [yColumn]:
        del df[col]
        
    return df, y

In [305]:
def regularize(X, r=0.000000001):
    return X + np.eye(X.shape[0]) * r

In [306]:
# def regularize(X, r: float = 0.00000001):
#     return X if r == 1 else X + np.eye(X.shape[0]) * r
#     
# X = [
#     [1, 2, 2],
#     [2, 1, 1], # r
#     [2, 1, 1], # r
#     [0, 9, 9],
#     [1, 0, 0],
# ] #     c  c
# X = np.array(X)
# XTX = X.T.dot(X)
# print(XTX) # duplicate columns is an issue for linear / logistic regression
# np.linalg.inv(XTX)
# 
# regularize(X, r=0.01)

In [307]:
def sigmoid(score):
    return 1 / (1 + np.exp(-score))

# z = np.linspace(-5, 5, 51)
# plt.plot(z, sigmoid(z))

In [308]:
def display_predictive_features_for_target(df: pd.DataFrame, target: str, categorical: Iterable[str] = []):
    global_target = df[target].mean()
    for c in categorical:
        df_group = df.groupby(c)[target].agg('mean','count')
        df_group['diff'] = df_group.mean - global_target
        df_group['risk'] = df_group.mean / global_target
        display(df_group)

In [309]:
def one_hot_encode(df: pd.DataFrame, dv: DictVectorizer = DictVectorizer(sparse=False), drop: Iterable[str] = [], fit: bool = False):
    assert set(df.columns).issuperset(drop), f'At least one of {drop} is not found in the DataFrame `df`'
    
    df_encode = df.copy()
    for feature in drop:
        del df_encode[feature]
    
    data = df_encode.to_dict(orient='records')
    X = dv.fit_transform(data) if fit else dv.transform(data)
        
    assert len(dv.feature_names_) == X.shape[1]
    return X, dv

In [310]:
def fit(model: skFit, df: pd.DataFrame, y: pd.Series, dv: DictVectorizer = DictVectorizer(sparse=False), drop: Iterable[str] = []) -> tuple[skFit, DictVectorizer]:
    assert df.shape[0] == y.shape[0], '`df` and `y` mismatch'
    
    X, dv = one_hot_encode(df, dv, drop, fit=True)
    model.fit(X, y)
    
    return model, dv

In [311]:
def decide(model: skDecide, dv: DictVectorizer, df: pd.DataFrame, drop: Iterable[str] = []):
    X, _ = one_hot_encode(df, dv, drop)
    y_pred = model.predict(X)
    
    return y_pred

In [312]:
def predict(model: skPredict, dv: DictVectorizer, df: pd.DataFrame, drop: Iterable[str] = []):
    X, _ = one_hot_encode(df, dv, drop)
    y_pred = model.predict_proba(X)[:, 1]
    
    return y_pred

In [313]:
def random_predictions(y: pd.Series, seed: int = 42):
    np.random.seed(seed)
    if y.dtype == 'bool':
        return np.random.uniform(0, 1, size=len(y))
    else:
        return np.random.uniform(0, 1, size=len(y))

In [314]:
def model_metrics(y: pd.Series, y_pred: pd.Series, threshold: float = 0.5):
    assert len(y) == len(y_pred), "`y` and `y_pred` mismatch"
    assert 0 <= threshold and threshold <= 1, "invalid threshold"
    
    actual_positive = (y == 1)
    actual_negative = (y == 0)
    predicted_positive = (y_pred >= threshold)
    predicted_negative = (y_pred < threshold)
    
    true_positive = (predicted_positive & actual_positive).sum()
    false_positive = (predicted_positive & actual_negative).sum()
    false_negative = (predicted_negative & actual_positive).sum()
    true_negative = (predicted_negative & actual_negative).sum()
    
    # Accuracy = ratio of correct predictions -- false and positive 
    # accuracy = proportional_matrix[0,0] + proportional_matrix[1,1]
    accuracy = (true_positive + true_negative) / (true_positive + false_positive + false_negative + true_negative)
    
    # Precision = ratio of correct to positive predictions; higher is better
    precision = true_positive / (true_positive + false_positive)
    # Recall = ratio of correctly predicted positive; higher is better
    recall = true_positive / (true_positive + false_negative)
    f1 = 2 * (precision * recall) / (precision + recall)
    
    # TPR = ratio of true positives in all positives; higher is better
    # true_positive_rate = recall
    true_positive_rate  = true_positive / ( false_negative + true_positive)
    # FPR = ratio of false positives in all negatives; lower is better
    false_positive_rate = false_positive / (true_negative + false_positive)
    
    matrix = np.array([
        #   g(Xi) < t    |   g(Xi) >= t
        [ true_negative  , false_positive ], # y == 0
        [ false_negative , true_positive  ]  # y == 1
    ])
    
    proportional_matrix = (matrix / matrix.sum()).round(6)
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'tp': true_positive,
        'fp': false_positive,
        'fn': false_negative,
        'tn': true_negative,
        'tpr': true_positive_rate,
        'fpr': false_positive_rate,
        'confusion': matrix,
        'proportional_confusion': proportional_matrix
    }

In [315]:
def model_metrics_for_thresholds(y: pd.Series, y_pred: pd.Series, thresholds: Iterable[float]) -> pd.DataFrame:
    results = []
    for threshold in thresholds:
        metrics = model_metrics(y, y_pred, threshold)
        metrics['threshold'] = threshold
        results.append(metrics)
        
    df_metrics = pd.DataFrame(results)
    df_metrics.set_index('threshold', inplace=True)
    
    return df_metrics

## Data Preparation

In [316]:
df = pd.read_csv('./jamb_exam_results.csv')
df.columns = df.columns.str.lower().str.replace(' ', '_')
df.head()

,jamb_score,study_hours_per_week,attendance_rate,teacher_quality,distance_to_school,school_type,school_location,extra_tutorials,access_to_learning_materials,parent_involvement,it_knowledge,student_id,age,gender,socioeconomic_status,parent_education_level,assignments_completed
0,192,22,78,4,12.4,Public,Urban,Yes,Yes,High,Medium,1,17,Male,Low,Tertiary,2
1,207,14,88,4,2.7,Public,Rural,No,Yes,High,High,2,15,Male,High,NaN,1
2,182,29,87,2,9.6,Public,Rural,Yes,Yes,High,Medium,3,20,Female,High,Tertiary,2
3,210,29,99,2,2.6,Public,Urban,No,Yes,Medium,High,4,22,Female,Medium,Tertiary,1
4,199,12,98,3,8.8,Public,Urban,No,Yes,Medium,Medium,5,22,Female,Medium,Tertiary,1


### Drop features and NaN -> 0 (see homework instructions)

In [317]:
del df['student_id']
df.fillna(0, inplace=True)
        
df.head().T

,0,1,2,3,4
jamb_score,192,207,182,210,199
study_hours_per_week,22,14,29,29,12
attendance_rate,78,88,87,99,98
teacher_quality,4,4,2,2,3
distance_to_school,12.4,2.7,9.6,2.6,8.8
school_type,Public,Public,Public,Public,Public
school_location,Urban,Rural,Rural,Urban,Urban
extra_tutorials,Yes,No,Yes,No,No
access_to_learning_materials,Yes,Yes,Yes,Yes,Yes
parent_involvement,High,High,High,Medium,Medium


### Split the data

#### Training, Testing, Validation, & Full (Training + Validation)

In [318]:
df_val, df_test, df_train, df_full = validation_testing_training_full_split(df, seed=1)

nTotal = len(df)
nVal = len(df_val)
nTest = len(df_test)
nTrain = len(df_train)
nFull = len(df_full)

round(nVal/nTotal, 1), round(nTest/nTotal, 1), round(nTrain/nTotal, 1), round(nFull/nTotal,1), round(nTotal/nTotal)

(0.2, 0.2, 0.6, 0.8, 1)

#### Split out `y` (target feature) from all datasets 

In [319]:
df_val, y_val = y_split(df_val, yColumn='jamb_score')
df_test, y_test = y_split(df_test, yColumn='jamb_score')
df_train, y_train = y_split(df_train, yColumn='jamb_score')
df_full, y_full = y_split(df_full, yColumn='jamb_score')

assert df_val.shape[1] == df_test.shape[1] and df_test.shape[1] == df_train.shape[1] and df_train.shape[1] == df_full.shape[1]
assert len(y_val) == df_val.shape[0] and len(y_test) == df_test.shape[0] and len(y_train) == df_train.shape[0] and len(y_full) == df_full.shape[0]

In [320]:
df_train.head().T

,0,1,2,3,4
study_hours_per_week,20,11,31,29,28
attendance_rate,72,80,82,79,96
teacher_quality,3,2,1,1,2
distance_to_school,4.4,3.3,8.3,15.8,8.9
school_type,Public,Public,Public,Public,Private
school_location,Urban,Urban,Urban,Rural,Rural
extra_tutorials,No,Yes,Yes,Yes,Yes
access_to_learning_materials,Yes,Yes,Yes,Yes,Yes
parent_involvement,Medium,Medium,Low,Low,Medium
it_knowledge,Low,High,High,Low,Low


## Question 1: Training the model

In [321]:
model, dv = fit(
    model=DecisionTreeRegressor(max_depth=1), 
    dv=DictVectorizer(sparse=True),
    df=df_train, 
    y=y_train,
)
model, dv

(DecisionTreeRegressor(max_depth=1), DictVectorizer())

In [322]:
print(export_text(model, feature_names=dv.feature_names_))

|--- study_hours_per_week <= 18.50
|   |--- value: [155.24]
|--- study_hours_per_week >  18.50
|   |--- value: [188.59]



Which feature is used for splitting the data?

* `study_hours_per_week`  `<--`
* `attendance_rate`
* `teacher_quality`
* `distance_to_school`

## Question 2: Random Forest

In [323]:
model, dv = fit(
    model=RandomForestRegressor(n_estimators=10, random_state=1, n_jobs=-1), 
    dv=DictVectorizer(sparse=True),
    df=df_train, 
    y=y_train,
)
model, dv

(RandomForestRegressor(n_estimators=10, n_jobs=-1, random_state=1),
 DictVectorizer())

In [324]:
y_val_pred = decide(model, dv, df_val)
root_mean_squared_error(y_val, y_val_pred)

42.13724207871227

What's the RMSE of this model on validation?

* 22.13
* 42.13 `<--`
* 62.13
* 82.12

## Question 3: Random Forest Tuning (n_estimators)

In [359]:
scores = []
for estimators in tqdm( range(10, 201, 10) ):
    model, dv = fit(
        model=RandomForestRegressor(n_estimators=estimators, random_state=1, n_jobs=-1),
        dv=DictVectorizer(sparse=True),
        df=df_train,
        y=y_train,
    )
    
    y_val_pred = decide(model, dv, df_val)
    scores.append((estimators, root_mean_squared_error(y_val, y_val_pred)))
    
df_scores = pd.DataFrame(scores, columns=['n_estimators', 'rmse'])
df_scores

100%|██████████| 20/20 [02:09<00:00,  6.45s/it]


,n_estimators,rmse
0,10,42.137242
1,20,41.461215
2,30,41.106171
3,40,40.917194
4,50,40.852279
5,60,40.784281
6,70,40.677098
7,80,40.539333
8,90,40.504346
9,100,40.516805


After which value of `n_estimators` does RMSE stop improving?
Consider 3 decimal places for calculating the answer.

- 10
- 25
- 80  `<--`
- 200

## Question 4: Random Forest Tuning (n_estimators & max_depth)

In [350]:
scores = []
for depth in [10, 15, 20, 25]:
    for estimators in tqdm( range(10, 201, 10), desc=f'max_depth = {depth}' ):
        model, dv = fit(
            model=RandomForestRegressor(max_depth=depth, n_estimators=estimators, random_state=1, n_jobs=-1),
            dv=DictVectorizer(sparse=True),
            df=df_train,
            y=y_train,
        )
        
        y_val_pred = decide(model, dv, df_val)
        scores.append((depth, estimators, root_mean_squared_error(y_val, y_val_pred)))
    
df_scores = pd.DataFrame(scores, columns=['max_depth', 'n_estimators', 'rmse'])
df_scores.groupby(by='max_depth').mean()

max_depth = 25: 100%|██████████| 20/20 [02:08<00:00,  6.42s/it]


,n_estimators,rmse
max_depth,,
10,105.0,40.392498
15,105.0,40.735282
20,105.0,40.739734
25,105.0,40.787866


What's the best `max_depth`, using the mean RMSE?

* 10 `<--`
* 15
* 20
* 25

## Question 5: Feature Imporance

In [331]:
model, dv = fit(
    model=RandomForestRegressor(n_estimators=10, max_depth=20, random_state=1, n_jobs=-1),
    dv=DictVectorizer(sparse=True),
    df=df_train,
    y=y_train,
)

df_scores = pd.DataFrame(zip(dv.feature_names_, model.feature_importances_), columns=['feature', 'importance'])
df_scores.sort_values(by='importance', ascending=False)

,feature,importance
27,study_hours_per_week,0.248354
4,attendance_rate,0.149729
5,distance_to_school,0.136486
28,teacher_quality,0.082682
2,age,0.069311
3,assignments_completed,0.031517
24,socioeconomic_status=High,0.025714
17,parent_involvement=High,0.022919
10,it_knowledge=High,0.017719
15,parent_education_level=Secondary,0.016957


What's the most important feature (among these 4)? 

* `study_hours_per_week` `<--`
* `attendance_rate`
* `distance_to_school`
* `teacher_quality`

## Question 6: Gradient Boosting

In [358]:
for eta in [0.3, 0.1]:

    xgb_params = {
        'eta': eta, 
        'max_depth': 6,
        'min_child_weight': 1,
        
        'objective': 'reg:squarederror',
        'nthread': 2,
        
        'seed': 1,
        'verbosity': 1,
    }

    X_train, dv = one_hot_encode(df_train, dv=DictVectorizer(sparse=True), fit=True)
    X_val, _ = one_hot_encode(df_val, dv=dv)

    dX_train = xgb.DMatrix(X_train, label=y_train, feature_names=dv.feature_names_)
    dX_val = xgb.DMatrix(X_val, label=y_val, feature_names=dv.feature_names_)

    model = xgb.train(xgb_params, dX_train, num_boost_round=100)

    y_val_pred = model.predict(dX_val)
    
    display(f'{eta=} -> rmse={root_mean_squared_error(y_val, y_val_pred):.4f}')

'eta=0.3 -> rmse=43.4188'

'eta=0.1 -> rmse=41.0503'

Which eta leads to the best RMSE score on the validation dataset?

* 0.3
* 0.1 `<--`
* Both give equal value